# DATA MINING PROJECT: Analysis of a Supermarket’s Customers
## 1.1) Data Understanding: Semantics & Quality
### *Antonio Strippoli, Valerio Mariani*

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sn
from scipy import stats
from math import log, ceil
from natsort import natsorted
import matplotlib.pyplot as plt

pd.set_option('mode.chained_assignment', None)

In [ ]:
def plot(ax, filename="", figsize=(6.4, 4.8)):
    fig = plt.gcf()
    fig.set_size_inches(*figsize)
    plt.tight_layout()
    if filename:
        plt.savefig(f"../report/imgs/{filename}")
    plt.show()
    plt.close()

In [ ]:
# Load the dataset from the given file
df = pd.read_csv('customer_supermarket.csv', sep='\t', index_col=0, parse_dates=["BasketDate"])

In [ ]:
# Prints data's samples and informations,
# including the number of not null values for each columns
df.info()
print("")
df.head()

## Data Semantics
Perform some variable wise checks to understand the dataset.

In [ ]:
# PRE: Check if basket starting with 'C' all have quantity less than 0
# Results: only basket starting with 'C' have quantity less than 0
tmp = df[ (df["BasketID"].str.contains('C')) & (df["Qta"] > 0) ]
print("N. BasketID STARTING WITH 'C' AND WITH Qta > 0:", len(tmp))

tmp = df[ (df["Qta"] < 0) & ~(df["BasketID"].str.contains('C')) ]
tmp.dropna(subset=['CustomerID'], inplace=True)
print("N. BasketID NOT STARTING WITH 'C' AND WITH Qta < 0:", len(tmp))

In [ ]:
# Check if we have the same product inside the same basket
# Result: two cases, same price, different price
def check_duplicated_prods(x):
    if len(x) > 1 and x["Sale"].nunique() == 1:
        return x
    return None

tmp = df.groupby(['BasketID','BasketDate','ProdID']).apply(check_duplicated_prods).dropna()

print("SAMPLE OF BASKET WITH INCONSISTENT Qta:")
tmp

In [ ]:
# Check if with same BasketID we have different datetimes
# Results: change BasketDate to PurchaseDate
tmp = df.groupby(["BasketID"]).nunique()["BasketDate"].eq(1)
tmp = tmp[tmp == False]
print("INCONSISTENT BasketDates:", len(tmp))

In [ ]:
# Check if two customers happen to have the same BasketID
# Result: after removing duplicates no other wrong value found
tmp = df.groupby(["BasketID", "CustomerID"]).ngroups
print("N. BasketID-CustomerID COUPLES:", tmp)

tmp = df["BasketID"].nunique()
print("N. BasketID:", tmp)

tmp = df.dropna(subset=['CustomerID'])
tmp = tmp.groupby(["BasketID"]).nunique()["CustomerID"].eq(1)
tmp = tmp[tmp == False].index
print("INCONSITENT BasketID-CustomerID (after NaN removal):", len(tmp))

In [ ]:
# Check if discount are always alone in the basket
# Result: Almost always, only one time we have it together with Manual
tmp = df[
    df["BasketID"].isin(
        df[df['ProdID'] == "D"]["BasketID"]
)]
tmp = tmp[tmp["ProdID"] != "D"]
print("PRODUCTS IN THE SAME BASKET WITH DISCOUNT:\n")
tmp

In [ ]:
# Check if baskets only are numerical with an optional starting 'C' character
# Result: We found baskets starting with 'A', which however will be removed since they have sales less than 0
tmp = df[~df['BasketID'].str.contains('C')][df['BasketID'].str.contains('[A-Za-z]')]["BasketID"].unique()
print("STRANGE BASKETS:")
tmp

In [ ]:
# Check for strange ProductID (nor alphanumerical code only)
# Result: A lot of products contains characters, we get to know about discounts and bank charges
tmp = df[df['ProdID'].str.contains('[A-Za-z]')]["ProdID"].unique()
print("STRANGE ProductID:\n", tmp)

In [ ]:
# Check for non-uppercase descriptions
# Result: we get to know about descriptions being inconsistent and some strange descriptions
tmp = df['ProdDescr'].isna().sum()
print("N. NaN ProdDescr:", tmp)

tmp = df.dropna(subset=['ProdDescr'])
tmp = tmp[tmp['ProdDescr'].str.contains('[a-z]')]["ProdDescr"].unique()
print("INCONSISTENT ProdDescr:")
tmp

In [ ]:
# Check list of countries
# Result: (Get to know about hidden null-values: 'Unspecified')
tmp = list(sorted(list(df["CustomerCountry"].unique())))
print("COUNTRIES:")
tmp

In [ ]:
# Check for strange qta values
# Result: Get to know about negative values and outliers
tmp = df['Qta'].describe()
print("Qta Descr:")
tmp

In [ ]:
# CustomerCountry seems like the country where the user registered... is that true?
# Result: no, since some IDs have different countries. Some customers may have changed their nationality.
# We will take this into account when we will create the customer profilation dataset.
tmp = df.groupby(["CustomerID"]).nunique()["CustomerCountry"].eq(1)
tmp = list(tmp[tmp == False].index)
print("INCONSISTENT CustomerCountry:")
tmp

In [ ]:
# Do all ProdID have one ProdDescr?
# Result: No, some descriptions are more verbose, we will take those
tmp = df.groupby(["ProdID"]).nunique()["ProdDescr"].eq(1)
tmp = tmp[tmp == False].index
print("N. INCONSISTENT ProdDescr:", len(tmp))

In [ ]:
# Do we have sales with more than 3 digit places?
# Result: Yes, we will round them
tmp = df["Sale"].astype(str).str.contains(r",\d{3,}")
tmp = tmp[tmp == True].index
tmp = df.loc[tmp]
print("INCONSISTENT Sale:")
tmp

## Data Quality
Clean up the datas by correcting semantical errors, removing outliers and other mixed fixes.

In [ ]:
# Converts sale to float, accomodating the csv format
df["Sale"] = df["Sale"].str.replace(',', '.').astype(float)

# Remove unidentified customers and converts CustomerID to int
df.dropna(subset=['CustomerID'], inplace=True)
df["CustomerID"] = df["CustomerID"].astype(int)

# Put all characters in uppercase and remove extra whitespaces for products' description
df["ProdDescr"] = df["ProdDescr"].str.upper().str.strip()

# Put all characters in uppercase for product ids
df["ProdID"] = df["ProdID"].str.upper()

# Remove purchases with prices less than or equal to zero, together with some outliers that costs less than 0.01
# We remove them since they're few (4)
df = df[df["Sale"] >= 0.01]

# Remove C from basketIDs, since it is pointless (we already have negative quantities to identify those)
# NOTE: We also previously dropped baskets starting with 'A', which had negative sale
df["BasketID"] = df["BasketID"].str.replace('C', '').astype(int)

In [ ]:
# Uniform descriptions of same productIDs by taking the longest (more informations)
tmp = df.groupby(["ProdID"]).nunique()["ProdDescr"].eq(1)
tmp = tmp[tmp == False].index
new_prod_descr = df[df["ProdID"].isin(tmp)].groupby("ProdID").agg({'ProdDescr': 'max'})

def uniform_descr(x):
    if x.loc["ProdID"] in new_prod_descr.index:
        descr = new_prod_descr.loc[x.loc["ProdID"]]["ProdDescr"]
        x.loc["ProdDescr"] = descr
    return x

df[["ProdID", "ProdDescr"]] = df[["ProdID", "ProdDescr"]].apply(uniform_descr, axis=1)

In [ ]:
# Put multiple products in the same basket as a single product
df = df.groupby(['BasketID','ProdID']).agg({
    'BasketDate': 'min',
    'Qta': 'sum',
    'Sale': 'mean',
    'CustomerID': 'min',
    'CustomerCountry': 'min',
    'ProdDescr': 'min'
}).reset_index()

In [ ]:
# Drop rows corresponding to returns without relative purchase (inconsistent data)
invalid_indexes = []
def get_invalid_indexes(x):
    x = x.sort_values(by='BasketDate')
    s = 0
    for i, qta in enumerate(x['Qta']):
        if (s := s + qta) < 0:
            invalid_indexes.append(x.iloc[i].name)
            s = 0

df[ ~df["ProdID"].isin(['M', 'D', 'BANK CHARGES']) ].groupby(['CustomerID', 'ProdID']).apply(get_invalid_indexes)
df.drop(invalid_indexes, inplace=True)

### OUTLIERS REMOVAL

In [ ]:
# Utility function
def iqr_non_outliers(s: pd.Series):
    """Returns a true-list of the outliers in a column
    of the DataFrame, based on the quantiles"""
    Q1 = s.quantile(0.25)
    Q3 = s.quantile(0.75)

    IQR = Q3 - Q1
    trueList = (s < (Q1 - 1.5 * IQR)) | (s > (Q3 + 1.5 * IQR))
    return trueList

### Outliers in ATTRIBUTES

In [ ]:
# Outliers in ATTRIBUTES from QTA
df_qta = df["Qta"]

plot(df_qta.plot.box(), figsize=(2, 4.8), filename="Outliers_Articles_Qta_BP")
plot(df_qta[abs(df_qta) < 100].hist(bins=100), filename="Outliers_Articles_Qta_HIST")

# Would IQR be effective?
# Result: no, since we think that most of the customers are wholesalers and it would drop too many entries
iqr_outliers = df_qta[~iqr_non_outliers(df_qta)]
print("QTA - IQR RESULTS:\n", iqr_outliers.describe())
print("MIN Qta Positives:", iqr_outliers[iqr_outliers > 0].min())
print("MAX Qta Negatives:", iqr_outliers[iqr_outliers < 0].max())

# Search for a threshold
plot(df_qta[abs(df_qta) < 10000].plot.box(), figsize=(2, 4.8))
plot(df_qta[(df_qta < 3500) & (df_qta > -2000)].plot.box(), figsize=(2, 4.8))

# One last check: how are those outliers distributed among the users?
outliers_i = df_qta[(df_qta > 3500) | (df_qta < -2000)].index
outliers = df.loc[outliers_i]
print("QTA OUTLIERS (with threshold of 3500):")
print(outliers["Qta"].describe())
print(outliers["CustomerID"].nunique())

# Values come from different users, we cannot just drop the customers, must instead drop single tuples
df.drop(outliers_i, inplace=True)

In [ ]:
# Outliers in ATTRIBUTES from SALE
df_sale = df['Sale']

plot(df_sale.plot.box(), figsize=(2, 4.8), filename="Outliers_Articles_Sale_BP")
plot(df_sale[df_sale < 50].hist(bins=100), filename="Outliers_Articles_Sale_HIST")

# Search for a threshold and remove based on that
plot(df_sale[df_sale < 5000].plot.box(), figsize=(2, 4.8))
plot(df_sale[df_sale < 2200].plot.box(), figsize=(2, 4.8))
df = df[df_sale < 2200]

### Outliers in BASKETS

In [ ]:
# Outliers in BASKETS from QTA
df_basket_qta = df[["BasketID", "Qta"]].groupby("BasketID").agg('sum')["Qta"]

plot(df_basket_qta.plot.box(), figsize=(2, 4.8), filename="Outliers_Basket_Sale_BP")
plot(df_basket_qta[abs(df_basket_qta) < 2000].hist(bins=100), filename="Outliers_Basket_Sale_HIST")

In [ ]:
# Remove in BASKETS from SALE
df_basket_cost = df[['BasketID', 'Qta', 'Sale']]
df_basket_cost['Cost'] = df_basket_cost['Qta'] * df_basket_cost['Sale']
df_basket_cost = df_basket_cost[["BasketID", "Cost"]].groupby("BasketID").agg('sum')["Cost"]

plot(df_basket_cost.plot.box(), figsize=(2, 4.8), filename="Outliers_Basket_Sale_BP")
plot(df_basket_cost[(df_basket_cost > -2000) & (df_basket_cost < 6000)].hist(bins=100), filename="Outliers_Basket_Sale_HIST")

# Check if IQR 
iqr_outliers = df_basket_cost[~iqr_non_outliers(df_basket_cost)]
print("BASKETID - IQR RESULTS:\n", iqr_outliers.describe())
print("MIN BASKETID-COST Postives:", iqr_outliers[iqr_outliers > 0].min())
print("MAX BASKETID-COST Negatives:", iqr_outliers[iqr_outliers < 0].max())

# Search for a threshold
plot(df_basket_cost[(df_basket_cost > -8000) & (df_basket_cost < 30000)].plot.box(), figsize=(2, 4.8), filename="Outliers_Basket_Sale_BP")

# One last check: how are those outliers distributed among the users?
outliers = df_basket_cost[(df_basket_cost <= -8000) | (df_basket_cost >= 30000)].index
customer_outliers = df[df['BasketID'].isin(outliers)]['CustomerID'].unique()
print("BASKETID OUTLIERS (WITH THRESHOLD)")
print("Baskets outliers:", len(outliers))
print("Customers having those baskets:", len(customer_outliers))

# Values come from different users, we cannot just drop the customers, must instead drop single tuples
df = df[~df["BasketID"].isin(outliers)]

### Outliers in Customers

In [ ]:
# Remove in CUSTOMERID from Total Items Purchased
df_customer_qta = df[['CustomerID', 'Qta']].groupby('CustomerID').agg('sum')

plot(df_customer_qta.plot.box(), figsize=(2, 4.8), filename="Outliers_Customer_Qta_BP")
plot(df_customer_qta[df_customer_qta < 25000].hist(bins=100), filename="Outliers_Customer_Qta_HIST")

# Search a threshold
plot(df_customer_qta[df_customer_qta < 100000].plot.box(), figsize=(2, 4.8), filename="Outliers_Customer_Qta_BP")
plot(df_customer_qta[df_customer_qta < 70000].plot.box(), figsize=(2, 4.8), filename="Outliers_Customer_Qta_BP")

non_outliers = df_customer_qta[df_customer_qta < 70000].dropna().index
df = df[df['CustomerID'].isin(non_outliers)]

In [ ]:
# Remove in CUSTOMERID from Profit
df_customer_sale = pd.Series([round( sum( g[1]["Sale"]*g[1]["Qta"] ), 2) for g in df.groupby('CustomerID')], index=[g[0] for g in df.groupby('CustomerID')], name="Profit")

plot(df_customer_sale.plot.box(), figsize=(2, 4.8), filename="Outliers_Customer_Sale_BP")
plot(df_customer_sale[df_customer_sale < 25000].hist(bins=100), filename="Outliers_Basket_Sale_HIST")

# Search a threshold
plot(df_customer_sale[df_customer_sale < 80000].plot.box(), figsize=(2, 4.8), filename="Outliers_Customer_Sale_BP")

non_outliers = df_customer_sale[df_customer_sale < 80000].dropna().index
df = df[df['CustomerID'].isin(non_outliers)]

### Minor final changes and save the dataset as a secondary data

In [ ]:
# Rename columns with names that could mislead
df.rename(columns={'BasketDate': 'PurchaseDate'}, inplace=True)

# Swap columns
df = df[["BasketID", "ProdID", "ProdDescr", "Sale", "Qta", "PurchaseDate", "CustomerID","CustomerCountry"]]

# Sort by date the dataset and reset indexes
df.sort_values("PurchaseDate", inplace=True)
df.reset_index(drop=True, inplace=True)

# Save the pre-processed dataset
df.to_csv("customer_supermarket_2.csv")